In [1]:
# 基本ツール
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
import matplotlib.ticker as ticker
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification
from sklearn import linear_model

from sklearn.metrics import mean_squared_error
# grid serch

from sklearn.model_selection import GridSearchCV

In [59]:
raw_data   = pd.read_csv('./train.csv')
raw_data_  = pd.read_csv('./test.csv')
sample_submission  = pd.read_csv('./sample_submission.csv')
train_data = raw_data.copy()
test_data  = raw_data_.copy()

In [60]:
#my_report = sweetviz.compare([train_data, 'Train'], [test_data, 'Test'], 'target')
#my_report.show_html('Report.html')

In [61]:
train_data.head(100)

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,A,B,A,A,B,D,A,E,C,...,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
1,2,B,A,A,A,B,B,A,E,A,...,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
2,3,A,A,A,C,B,D,A,B,C,...,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
3,4,A,A,A,C,B,D,A,E,G,...,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
4,6,A,B,A,A,B,B,A,E,C,...,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,164,A,B,A,C,B,B,A,E,E,...,0.198953,0.304955,0.228038,0.165184,0.350502,0.231549,0.265881,0.351718,0.251234,8.574307
96,165,A,B,A,D,B,B,A,E,C,...,0.390233,0.406308,0.254529,0.401283,0.443222,0.611084,0.373161,0.329228,0.289662,7.613036
97,167,A,B,A,C,B,D,A,E,E,...,0.435150,0.340275,0.260918,0.311286,0.611101,0.568261,0.397329,0.650514,0.805869,7.691082
98,168,A,B,A,C,C,D,A,E,G,...,0.679719,0.480595,0.484355,0.913394,0.840664,0.577120,0.346455,0.893273,0.671947,8.441794


In [63]:
X = train_data.iloc[:, 1:25]
y = train_data.iloc[:,25:]

X_test = test_data.iloc[:,1:25]

In [67]:
#　ダミー化
X = pd.get_dummies(X, columns=['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9'])
X_test = pd.get_dummies(X_test, columns=['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9'])


In [68]:
# 正規化
X = (X - np.min(X)) / (np.max(X) - np.min(X))
y = (y - np.min(y)) / (np.max(y) - np.min(y))

In [70]:
X = X.iloc[:, :69]

           cont0     cont1     cont2     cont3     cont4     cont5     cont6  \
0       0.887037  0.816091  0.171367  0.095360  0.114564  0.855691  0.359931   
1       0.463397  0.076438  0.387501  0.816060  0.115723  0.465907  0.288058   
2       0.720411  0.898964  0.477677  0.778047  0.129890  0.884880  0.310331   
3       0.696796  0.911708  0.198531  0.686162  0.721352  0.902538  0.509813   
4       0.505656  0.765785  0.516239  0.251719  0.111686  0.415176  0.630849   
...          ...       ...       ...       ...       ...       ...       ...   
299995  0.309047  0.846382  0.205883  0.353368  0.719690  0.315305  0.204602   
299996  0.232781  0.194846  0.605575  0.067627  0.729014  0.251359  0.203048   
299997  0.380712  0.742099  0.485420  0.325038  0.134425  0.474276  0.326122   
299998  0.604542  0.817289  0.668978  0.060156  0.707507  0.397972  0.267000   
299999  0.496397  0.102293  0.755170  0.302129  0.629733  0.726317  0.343249   

           cont7     cont8     cont9  .

In [30]:
# random forest

rf_model = RandomForestRegressor(random_state=1)

# max_fature　決定木作成に用いる変数の数

features = [30]

# n_estimators 決定木の数

estimates = [50]

# min_sample_split　分岐を続行するためのサンプルサイズ

min_sample = [300]

# max_depth 

depthes = [50]


# 合計パラメータ

params = {'n_estimators':estimates, 'max_features':features, 'min_samples_split':min_sample, 'max_depth':depthes}

In [33]:
# 交差検証

grid_cv = GridSearchCV(estimator=rf_model, param_grid=params, cv=5)


In [35]:
# X トレーニングデータ　y　はターゲット

grid_cv.fit(X,y.values.ravel())

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=1,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jobs=

In [36]:
display(grid_cv.best_params_,grid_cv.best_score_)

{'max_depth': 50,
 'max_features': 30,
 'min_samples_split': 300,
 'n_estimators': 50}

0.07436097415523628

In [37]:
forest_grid_pred = grid_cv.best_estimator_.predict(X)
forest_grid_mse = mean_squared_error(y, forest_grid_pred)
forest_grid_rmse = np.sqrt(forest_grid_mse)
forest_grid_rmse

0.0790876276936703

In [52]:
pred = grid_cv.best_estimator_.predict(X_test)

In [55]:
# 提出

sample_submission['target'] = pred


In [56]:
# csvファイル書きだし

sample_submission.to_csv('sample_submission.csv', index=False)